# 📚 บทที่ 2.1: Data Loading - การโหลดข้อมูล

## 🎯 เรื่องราวของนักวิทยาศาสตร์ข้อมูล

สมมติว่าคุณเป็นนักวิทยาศาสตร์ข้อมูลที่ต้องทำงาน 3 โปรเจกต์พร้อมกัน:

1. **โปรเจกต์ Customer Analysis** - ข้อมูลมาจาก **CSV file**
2. **โปรเจกต์ Product Recognition** - ข้อมูลมาจาก **Image files**
3. **โปรเจกต์ Real-time Weather Prediction** - ข้อมูลมาจาก **API**

มาเรียนรู้วิธีโหลดข้อมูลจากแต่ละแหล่งกันเลย!

In [ ]:
# Import libraries ที่จำเป็น
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import requests
import json
import os
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Libraries loaded successfully!")

---

## 📊 Part 1: Loading CSV Data

### เรื่องราว: บริษัทค้าปลีกต้องการวิเคราะห์ข้อมูลลูกค้า

ข้อมูลลูกค้าถูกเก็บในไฟล์ CSV มี:
- Customer ID
- Age (อายุ)
- Income (รายได้)
- Spending Score (คะแนนการใช้จ่าย)
- Product Category (หมวดสินค้าที่ชอบ)

In [ ]:
# สร้าง Mock Data สำหรับ Customer Analysis
np.random.seed(42)

n_customers = 500

customer_data = {
    'CustomerID': [f'CUST{str(i).zfill(4)}' for i in range(1, n_customers + 1)],
    'Age': np.random.randint(18, 70, n_customers),
    'Income': np.random.randint(20000, 150000, n_customers),
    'SpendingScore': np.random.randint(1, 100, n_customers),
    'Category': np.random.choice(['Electronics', 'Fashion', 'Food', 'Sports', 'Books'], n_customers)
}

df = pd.DataFrame(customer_data)

# บันทึกเป็น CSV
df.to_csv('customer_data.csv', index=False)
print("✅ Created customer_data.csv")
print(f"📊 Total customers: {len(df)}")

In [ ]:
# โหลดข้อมูล CSV
df_loaded = pd.read_csv('customer_data.csv')

print("📂 CSV File Loaded!")
print(f"\n📏 Shape: {df_loaded.shape}")
print(f"👥 Rows: {df_loaded.shape[0]}")
print(f"📋 Columns: {df_loaded.shape[1]}")
print(f"\n🔍 Column names: {list(df_loaded.columns)}")
print(f"\n📊 Data types:")
print(df_loaded.dtypes)

In [ ]:
# ดูข้อมูลตัวอย่าง
print("👀 First 10 customers:")
df_loaded.head(10)

In [ ]:
# สถิติเบื้องต้น
print("📈 Statistical Summary:")
df_loaded.describe()

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Age Distribution
axes[0, 0].hist(df_loaded['Age'], bins=20, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Age Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Frequency')

# Income Distribution
axes[0, 1].hist(df_loaded['Income'], bins=20, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Income Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Income ($)')
axes[0, 1].set_ylabel('Frequency')

# Category Distribution
category_counts = df_loaded['Category'].value_counts()
axes[1, 0].bar(category_counts.index, category_counts.values, color='coral')
axes[1, 0].set_title('Product Category Preferences', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Category')
axes[1, 0].set_ylabel('Count')
axes[1, 0].tick_params(axis='x', rotation=45)

# Spending Score vs Income
axes[1, 1].scatter(df_loaded['Income'], df_loaded['SpendingScore'], alpha=0.6, color='purple')
axes[1, 1].set_title('Spending Score vs Income', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Income ($)')
axes[1, 1].set_ylabel('Spending Score')

plt.tight_layout()
plt.show()

print("✅ CSV data visualization complete!")

---

## 🖼️ Part 2: Loading Image Data

### เรื่องราว: ร้านค้าออนไลน์ต้องการระบบจำแนกสินค้า

เรามีภาพสินค้า 3 ประเภท:
- **Shoes** (รองเท้า)
- **Shirts** (เสื้อ)
- **Bags** (กระเป๋า)

จะโหลดภาพมาเตรียมสำหรับ Deep Learning อย่างไร?

In [ ]:
# สร้าง Mock Images (สีต่างกันแทนภาพจริง)
import os
from PIL import Image

# สร้างโฟลเดอร์
os.makedirs('product_images/shoes', exist_ok=True)
os.makedirs('product_images/shirts', exist_ok=True)
os.makedirs('product_images/bags', exist_ok=True)

# สร้างภาพ mock (ใช้สีแทน)
def create_mock_images(category, color, num_images=5):
    for i in range(num_images):
        # สร้างภาพสีเดียว 224x224
        img = Image.new('RGB', (224, 224), color=color)
        img.save(f'product_images/{category}/{category}_{i+1}.jpg')

# สร้างภาพแต่ละหมวด
create_mock_images('shoes', (139, 69, 19), 5)    # Brown
create_mock_images('shirts', (0, 100, 200), 5)   # Blue
create_mock_images('bags', (200, 50, 50), 5)     # Red

print("✅ Created mock product images")
print("📁 Folder structure:")
print("   product_images/")
print("   ├── shoes/ (5 images)")
print("   ├── shirts/ (5 images)")
print("   └── bags/ (5 images)")

In [ ]:
# วิธีที่ 1: โหลดภาพทีละภาพ
from PIL import Image
import numpy as np

# โหลดภาพ 1 ภาพ
image_path = 'product_images/shoes/shoes_1.jpg'
img = Image.open(image_path)

print("📸 Image loaded!")
print(f"   Format: {img.format}")
print(f"   Size: {img.size}")
print(f"   Mode: {img.mode}")

# แปลงเป็น numpy array
img_array = np.array(img)
print(f"\n🔢 NumPy array shape: {img_array.shape}")
print(f"   Height: {img_array.shape[0]}")
print(f"   Width: {img_array.shape[1]}")
print(f"   Channels (RGB): {img_array.shape[2]}")
print(f"   Data type: {img_array.dtype}")
print(f"   Min value: {img_array.min()}")
print(f"   Max value: {img_array.max()}")

# แสดงภาพ
plt.figure(figsize=(5, 5))
plt.imshow(img)
plt.title('Product Image: Shoe', fontsize=14, fontweight='bold')
plt.axis('off')
plt.show()

In [ ]:
# วิธีที่ 2: โหลดหลายภาพพร้อมกัน
import glob

def load_images_from_folder(folder_path, target_size=(224, 224)):
    """
    โหลดภาพทั้งหมดจากโฟลเดอร์
    """
    images = []
    labels = []
    
    # หา categories
    categories = os.listdir(folder_path)
    
    for idx, category in enumerate(categories):
        category_path = os.path.join(folder_path, category)
        if not os.path.isdir(category_path):
            continue
            
        print(f"Loading {category}...")
        
        # โหลดภาพในแต่ละ category
        image_files = glob.glob(os.path.join(category_path, '*.jpg'))
        
        for img_path in image_files:
            img = Image.open(img_path)
            img = img.resize(target_size)
            img_array = np.array(img)
            
            images.append(img_array)
            labels.append(idx)
    
    return np.array(images), np.array(labels), categories

# โหลดข้อมูล
X_images, y_labels, categories = load_images_from_folder('product_images')

print(f"\n✅ Loaded all images!")
print(f"📊 Total images: {len(X_images)}")
print(f"🏷️ Categories: {categories}")
print(f"📐 Image array shape: {X_images.shape}")
print(f"   (samples, height, width, channels)")

In [ ]:
# แสดงภาพตัวอย่างจากแต่ละหมวด
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, category in enumerate(categories):
    # หาภาพแรกของแต่ละ category
    sample_idx = np.where(y_labels == idx)[0][0]
    
    axes[idx].imshow(X_images[sample_idx])
    axes[idx].set_title(f'{category.capitalize()}', fontsize=14, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("✅ Image data loaded successfully!")

---

## 🌐 Part 3: Loading Data from API

### เรื่องราว: ระบบทำนายสภาพอากาศต้องการข้อมูล Real-time

เราจะโหลดข้อมูลจาก API (ตัวอย่างใช้ Mock API)

In [ ]:
# สร้าง Mock API response
mock_weather_data = {
    "location": "Bangkok",
    "current": {
        "temperature": 32.5,
        "humidity": 75,
        "wind_speed": 12.3,
        "condition": "Partly Cloudy"
    },
    "forecast": [
        {"day": "Monday", "temp_max": 34, "temp_min": 26, "rain_chance": 30},
        {"day": "Tuesday", "temp_max": 33, "temp_min": 25, "rain_chance": 60},
        {"day": "Wednesday", "temp_max": 31, "temp_min": 24, "rain_chance": 80},
        {"day": "Thursday", "temp_max": 32, "temp_min": 25, "rain_chance": 40},
        {"day": "Friday", "temp_max": 33, "temp_min": 26, "rain_chance": 20}
    ]
}

# บันทึกเป็น JSON file (จำลอง API response)
with open('weather_api_response.json', 'w') as f:
    json.dump(mock_weather_data, f, indent=2)

print("✅ Created mock API response")

In [ ]:
# โหลดข้อมูลจาก JSON (จำลอง API call)
with open('weather_api_response.json', 'r') as f:
    weather_data = json.load(f)

print("🌤️ Weather Data Loaded from API")
print(f"\n📍 Location: {weather_data['location']}")
print(f"\n🌡️ Current Conditions:")
print(f"   Temperature: {weather_data['current']['temperature']}°C")
print(f"   Humidity: {weather_data['current']['humidity']}%")
print(f"   Wind Speed: {weather_data['current']['wind_speed']} km/h")
print(f"   Condition: {weather_data['current']['condition']}")

print(f"\n📅 5-Day Forecast:")
for forecast in weather_data['forecast']:
    print(f"   {forecast['day']}: {forecast['temp_min']}-{forecast['temp_max']}°C, Rain: {forecast['rain_chance']}%")

In [ ]:
# แปลงเป็น DataFrame สำหรับ Machine Learning
forecast_df = pd.DataFrame(weather_data['forecast'])

print("📊 Forecast DataFrame:")
print(forecast_df)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Temperature Forecast
x_pos = np.arange(len(forecast_df))
axes[0].bar(x_pos - 0.2, forecast_df['temp_max'], 0.4, label='Max Temp', color='orangered')
axes[0].bar(x_pos + 0.2, forecast_df['temp_min'], 0.4, label='Min Temp', color='skyblue')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(forecast_df['day'])
axes[0].set_ylabel('Temperature (°C)')
axes[0].set_title('5-Day Temperature Forecast', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Rain Chance
axes[1].plot(forecast_df['day'], forecast_df['rain_chance'], marker='o', linewidth=2, markersize=8, color='blue')
axes[1].fill_between(range(len(forecast_df)), forecast_df['rain_chance'], alpha=0.3, color='blue')
axes[1].set_ylabel('Rain Chance (%)')
axes[1].set_title('Rain Probability', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ API data visualization complete!")

---

## 📝 สรุป: Data Loading

### 🎯 สิ่งที่เราได้เรียนรู้:

#### 1. **CSV Data**
- ใช้ `pd.read_csv()` โหลดข้อมูลตาราง
- ตรวจสอบ shape, columns, data types
- ใช้ `.describe()` ดูสถิติเบื้องต้น
- Visualize ด้วย histogram, scatter plot

#### 2. **Image Data**
- ใช้ `PIL.Image.open()` โหลดภาพ
- แปลงเป็น numpy array สำหรับ Deep Learning
- Shape: (height, width, channels)
- โหลดหลายภาพพร้อมกันด้วย loop

#### 3. **API Data**
- โหลดข้อมูล JSON จาก API
- แปลงเป็น DataFrame
- Visualize ข้อมูล time series

### 🚀 Next Step:

ตอนนี้เรามีข้อมูลแล้ว แต่ข้อมูลยังไม่สะอาด!

👉 [บทต่อไป: Data Cleaning (การทำความสะอาดข้อมูล)](02_data_cleaning.ipynb)

เราจะเรียนรู้:
- จัดการ Missing Values
- หา Outliers
- แก้ไข Inconsistencies

---

## 💡 Tips & Best Practices

### ✅ Do:
- ตรวจสอบ data shape และ types เสมอ
- ดูตัวอย่างข้อมูลก่อนใช้งาน (`.head()`)
- Visualize ข้อมูลเพื่อเข้าใจ distribution
- บันทึก data loading pipeline ให้เป็นฟังก์ชัน

### ❌ Don't:
- อย่าโหลดข้อมูลทั้งหมดเข้า memory ถ้ามีขนาดใหญ่มาก
- อย่าลืมตรวจสอบ file path
- อย่ากด Run ทุก cell พร้อมกันถ้าใช้เวลานาน